load in base and finetuned model for captioning

In [ ]:
!pip install -q transformers

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration

ft_processor = BlipProcessor.from_pretrained("Gurveer05/blip-image-captioning-base-rscid-finetuned")
ft_model = BlipForConditionalGeneration.from_pretrained("pruhtopia/blip-rsc-5k-1")

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/780 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [ ]:
base_processor = BlipProcessor.from_pretrained("Gurveer05/blip-image-captioning-base-rscid-finetuned")
base_model = BlipForConditionalGeneration.from_pretrained("Gurveer05/blip-image-captioning-base-rscid-finetuned")

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

load in clip embedding model

In [ ]:
!pip install -q huggingface_hub open_clip_torch
!git clone https://github.com/ChenDelong1999/RemoteCLIP/

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.3 MB/s eta 0:00:00
Cloning into 'RemoteCLIP'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 135 (delta 26), reused 11 (delta 11), pack-reused 97 (from 1)
Receiving objects: 100% (135/135), 5.02 MiB | 38.33 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [ ]:
from huggingface_hub import hf_hub_download
import torch, open_clip
from PIL import Image
from IPython.display import display

# for model_name in ['RN50'] #, 'ViT-B-32', 'ViT-L-14']: #faster loading
for model_name in ['RN50', 'ViT-B-32', 'ViT-L-14']: #all models
    checkpoint_path = hf_hub_download("chendelong/RemoteCLIP", f"RemoteCLIP-{model_name}.pt", cache_dir='checkpoints')
    print(f'{model_name} is downloaded to {checkpoint_path}.')

RemoteCLIP-RN50.pt:   0%|          | 0.00/408M [00:00<?, ?B/s]

RN50 is downloaded to checkpoints/models--chendelong--RemoteCLIP/snapshots/bf1d8a3ccf2ddbf7c875705e46373bfe542bce38/RemoteCLIP-RN50.pt.


RemoteCLIP-ViT-B-32.pt:   0%|          | 0.00/605M [00:00<?, ?B/s]

ViT-B-32 is downloaded to checkpoints/models--chendelong--RemoteCLIP/snapshots/bf1d8a3ccf2ddbf7c875705e46373bfe542bce38/RemoteCLIP-ViT-B-32.pt.


RemoteCLIP-ViT-L-14.pt:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

ViT-L-14 is downloaded to checkpoints/models--chendelong--RemoteCLIP/snapshots/bf1d8a3ccf2ddbf7c875705e46373bfe542bce38/RemoteCLIP-ViT-L-14.pt.


In [ ]:
model_name = 'RN50'
clip_model, _, preprocess = open_clip.create_model_and_transforms(model_name)
processor = open_clip.get_tokenizer(model_name)

path_to_your_checkpoints = 'checkpoints/models--chendelong--RemoteCLIP/snapshots/bf1d8a3ccf2ddbf7c875705e46373bfe542bce38'

ckpt = torch.load(f"{path_to_your_checkpoints}/RemoteCLIP-{model_name}.pt", map_location="cpu")
message = clip_model.load_state_dict(ckpt)
clip_model = clip_model.eval()

<ipython-input-14-75d813f69610>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(f"{path_to_your_checkpoints}/RemoteCLIP-{model_name}.pt", map_location="cp

In [ ]:
def encode_image(clip_model, preprocess, pil_image):
    """
    Encodes a PIL image into feature embeddings using the CLIP model.

    Args:
        clip_model: The CLIP model.
        preprocess: The preprocessing function from open_clip.
        pil_image (PIL.Image.Image): The input image as a PIL image.

    Returns:
        torch.Tensor: Feature embeddings of the image.
    """
    # Ensure the image is in RGB mode
    if pil_image.mode != "RGB":
        pil_image = pil_image.convert("RGB")

    # Preprocess the image and add batch dimension
    image_tensor = preprocess(pil_image).unsqueeze(0).cuda()  # Add batch dimension and move to GPU

    # Generate image embeddings
    with torch.no_grad():
        image_features = clip_model.encode_image(image_tensor)

    return image_features

In [ ]:
def encode_text(clip_model, tokenizer, text_query):
    """
    Encodes a text query into feature embeddings using the CLIP model.

    Args:
        clip_model: The CLIP model.
        tokenizer: The tokenizer from open_clip.
        text_query (str): The text query to encode.

    Returns:
        torch.Tensor: Feature embeddings of the text query.
    """
    # Tokenize the text query
    text_tokens = tokenizer([text_query])  # Tokenize and batch the text

    # Generate text embeddings
    with torch.no_grad():
        text_features = clip_model.encode_text(text_tokens.cuda())  # Move tokens to GPU

    return text_features

In [ ]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("pruhtopia/rsc-5k-1", split="train")

# Extract the last 100 rows as the test dataset
test_dataset = dataset.select(range(len(dataset) - 100, len(dataset)))

README.md:   0%|          | 0.00/321 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/308M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/303M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5349 [00:00<?, ? examples/s]

In [ ]:
import torch.nn.functional as F
from IPython.display import display

def calculate_cos_similarity(embedding1, embedding2):
    """
    Calculate the cosine similarity between two embeddings.
    """
    return F.cosine_similarity(embedding1, embedding2, dim=-1).item()

# Initialize variables to track cosine similarities
base_model_cos_sim_total = 0
ft_model_cos_sim_total = 0
num_samples = len(test_dataset)

# Loop through the test dataset
for i, example in enumerate(test_dataset):
    # Get the image and text
    pil_image = example["image"]
    true_text = example["text"]

    # Display the image
    print(f"Sample {i + 1}:")
    display(pil_image)

    # Generate the base model description
    base_model_inputs = base_processor(images=pil_image, return_tensors="pt")
    base_model_outputs = base_model.generate(**base_model_inputs)
    base_model_description = base_processor.decode(base_model_outputs[0], skip_special_tokens=True)

    # Generate the fine-tuned model description
    ft_model_inputs = ft_processor(images=pil_image, return_tensors="pt")
    ft_model_outputs = ft_model.generate(**ft_model_inputs)
    ft_model_description = ft_processor.decode(ft_model_outputs[0], skip_special_tokens=True)

    # Display the descriptions
    print(f"True Text: {true_text}")
    print(f"Base Model Description: {base_model_description}")
    print(f"Fine-Tuned Model Description: {ft_model_description}")

    # Preprocess the image for the CLIP model
    with torch.no_grad():
        image_tensor = preprocess(pil_image).unsqueeze(0)  # Preprocess and add batch dimension
        image_embedding = clip_model.encode_image(image_tensor)

    # Tokenize the descriptions for the CLIP model
    with torch.no_grad():
        # Tokenize base model description
        base_text_tokens = processor([base_model_description])  # Tokenize text as a list
        base_text_tokens_tensor = torch.tensor(base_text_tokens).long()  # Convert to PyTorch tensor
        base_text_embedding = clip_model.encode_text(base_text_tokens_tensor)

        # Tokenize fine-tuned model description
        ft_text_tokens = processor([ft_model_description])  # Tokenize text as a list
        ft_text_tokens_tensor = torch.tensor(ft_text_tokens).long()  # Convert to PyTorch tensor
        ft_text_embedding = clip_model.encode_text(ft_text_tokens_tensor)

    # Calculate cosine similarities
    base_model_cos_sim = calculate_cos_similarity(image_embedding, base_text_embedding)
    ft_model_cos_sim = calculate_cos_similarity(image_embedding, ft_text_embedding)

    # Display cosine similarities
    print(f"Base Model Cosine Similarity: {base_model_cos_sim:.4f}")
    print(f"Fine-Tuned Model Cosine Similarity: {ft_model_cos_sim:.4f}")

    # Accumulate cosine similarities for averaging
    base_model_cos_sim_total += base_model_cos_sim
    ft_model_cos_sim_total += ft_model_cos_sim

    print("-" * 30)

# Calculate and display the average cosine similarities
base_model_avg_cos_sim = base_model_cos_sim_total / num_samples
ft_model_avg_cos_sim = ft_model_cos_sim_total / num_samples

print(f"Average Cosine Similarity for Base Model: {base_model_avg_cos_sim:.4f}")
print(f"Average Cosine Similarity for Fine-Tuned Model: {ft_model_avg_cos_sim:.4f}")

